In [2]:
import os
from os import listdir
from os.path import isfile, join, exists
import re
import collections
from dataclasses import dataclass
import json
from anytree import Node, RenderTree, AsciiStyle, PreOrderIter

In [3]:
###############################################################################
# REMatcher is a utility class to make regex's work similar to Perl
# NOTE: if there are any changes needed in this function contact Sean.
###############################################################################
class REMatcher(object):
    ''' This is a utility function to help out with matching regex's and
        grabbing the matches out in a way that is similar to how perl
        does it.
    '''
    def __init__(self, matchstring: str) -> None:
        self.matchstring = matchstring

    def match(self, regexp: str) -> bool:
        self.rematch = re.match(regexp, self.matchstring,
                                re.IGNORECASE |
                                re.MULTILINE |
                                re.DOTALL
                                )
        return bool(self.rematch)

    def group(self, i: int) -> str:
        return "Error 999" if self.rematch is None else self.rematch.group(i)

In [4]:
###############################################################################
# Creating a Data Class to hold the signal groups
###############################################################################
@dataclass
class SigGroup:
    title: str
    level: int
    sig_list: list[str]


In [18]:
# Main Dict
config_dict = collections.OrderedDict()

# Globals, these will be command line options
opt_gktw = '/home/sean/github/RTL_Design_Projects/common/common_force_tb/async_fifo/debug.gtkw'
opt_repl = ''
opt_config_path = '/home/sean/github/RTL_Design_Projects/common/common_force_tb/async_fifo/'

# not a command line option, but still kind of an option, eventually
opt_vcd_file = ''

# A list of dictionaries that gets created from the gtkw file
# TODO: figure out how to do generic encoding for command/control busses
sample_rate = -1
all_times_list = []
marker_list = []
tree_stack = [] # this is a a list of anytree nodes for grouping the signals
config_list = [] # as we iterate thru the gktw, and find and "End" we add it to the list
phase_0p2_signals = []
phase_0p8_signals = []
all_signals = []
off_list = [] # these are each of the "start" times in the gtkw
max_list = [] # these are each of the "end" times in the gtkw


In [19]:
# Read and Piece together the pieces from the config file(s)
# store in local vars first, then build up the dict's in the
with open(opt_gktw, 'r') as file:
    lines = file.readlines()

pos_found = False
line_count = len(lines)
curr_sig_group = None
curr_node = None

for i in range(line_count):
    line = lines[i]
    m = REMatcher(line)
    if m.match(r'^.dumpfile.\s+"(\S+)"'):
        opt_vcd_file = m.group(1)

    elif m.match(r'^.pos.\s+.*'):
        pos_found = True
        line_p1 = lines[i+1]
        atl = line_p1.split(' ')[2:]
        all_times_list = [x for x in atl if x != '-1'] # remove all of the -1 from the list

    elif m.match(r'^-sample_rate\s*=\s*(.*)\s*ns'):
        sample_rate = int(m.group(1))

    elif m.match(r'^-(.*)'): # spending a huge amount of time on this and I don't know if I'll use it :)
        key = m.group(1).rstrip()
        if '*' not in key:
            curr_group_key_list = [key]
            title = key
            level = 0
            curr_sig_group = SigGroup(title, level, [])
            curr_node = Node(curr_sig_group)
            tree_stack.append(curr_node)

        else:
            count = 0
            for char in key:
                if char == '*':
                    count += 1
                else:
                    break
            title = key
            level = count
            new_sig_group = SigGroup(title, level, [])
            if level > curr_sig_group.level:
                new_node = Node(new_sig_group, parent=curr_node)
            else:
                level_diff = curr_sig_group.level - level
                iter_node = curr_node
                for _ in range(level_diff):
                    iter_node = iter_node.parent
                curr_node = iter_node
                curr_node.parent.sig_list.append(new_sig_group)
                curr_sig_group = new_sig_group

    elif m.match(f'.markername.\s_.(.*)'):
        marker = m.group(1).rstrip()
        marker_list.append(marker)

    elif m.match(r'^\[') or m.match(r'^\@') or m.match(r'^\*'):
        continue

    else:
        full_sig = line.rstrip()
        all_signals.append(full_sig)
        curr_sig_group.sig_list.append(full_sig)
        sig_hier_list = full_sig.split('.')
        last_idx = len(sig_hier_list) -1
        sig = sig_hier_list[last_idx]
        sig_type = sig.split('_')[0]
        if  sig_type in ['i', 'o', 'r']:
            phase_0p2_signals.append(full_sig)

        elif sig_type in ['iw', 'ow', 'w']:
            phase_0p8_signals.append(full_sig)


In [21]:
# for node in tree_stack:
#     print(RenderTree(node, style=AsciiStyle))

In [ ]:
# Munge all of the data together into a dict
if sample_rate < 1:
    sample_rate = 10
len_atl = len(all_times_list)
len_ml = len(marker_list)
if len_ml != len_atl:
    print('Error: the number of markers does not mat the number of times')

In [ ]:
# Final Stage, write out the configuration file
i = 0
for cfg in config_list:
    file_name = f'{opt_config_path}{cfg}_{i}.json'
    with open(file_name, 'w') as file:
        json.dumps(cfg, file) 